In [5]:
from datetime import datetime
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#In order to gain the optimal use of this function, the user must have clean data: data lacking anomalies, such as nan
#or null types as values. Also, data must be comparable with each other such as ints, or strings. 

#This function is used to calculate the conditional likelihood of each column assuming that the table input has
#clean data. The chosen represents the column name which you would like to estimate. The returning statement is a 
#dictionary of values represented as each key named table + column number and their value representing the condiitonal
#likelihood for that column given chosen. There is also a prior key.
def conditional(chosen, table):
    y = []
    prior =[]
    holder = table[chosen]
    
    #Calculate the prior
    
    for x in table[chosen]:
        if x not in y:
            y.append(x)
    for x in y:
        table.where(table[chosen])
    for x in y:
        prior.append(table.where(chosen,x).num_rows/table.num_rows)
    
    #Calculate the condiitonal likelihoods
    table1 = table.drop(chosen)
    table2 = table1.with_column(chosen, holder)
    a={}
    t=0
    for x in range(0,table1.num_columns):
        b=[]
        for o in table1[x]:
            if o not in b:
                b.append(o)
        b = np.sort(b)
        p=[]
        q=0
        for w in b:
            p.append([])
            for z in y:
                p[q].append(table2.where(chosen,z).where(x,w).num_rows/table.where(chosen,z).num_rows)
            q+=1
        a['table'+str(t)]=p
        t+=1
    a['prior'] = prior
    return a

#Accuracy uses a "a" conditional likelihood dictionary, a table on which it will be calculating predictions and 
#comparing with, and chosen representing the estimated column variable.
def accuracy(a, table, chosen):
    p=[]
    b=[]
    table1 = table.drop(chosen)
    table2 = table1.with_column(chosen,table[chosen])
    for x in range(0,table2.num_columns):
        b.append([])
        for o in table2[x]:
            if o not in b[x]:
                b[x].append(o)
        b[x] = np.sort(b[x])
    for x in range(0, table.num_rows):
        prior = np.transpose(a['prior'])
        for y in range(0,table1.num_columns):
            t = get_index(b[y],table1.take(x)[y][0])
            prior=prior*a['table'+str(y)][t]
        p.append(b[len(b)-1][np.argmax(prior)])
    return np.count_nonzero(p==table[chosen])/table.num_rows

#Returns the index of a value, "y", in array, "array". However if "y" is not in "array", than return -1.
def get_index(array,y):
    t=0
    for x in array:
        if x==y:
            return t
        t+=1
    return -1   

#Combining accuracy and conditional, thus we are able to determine the training accuracy. By providing seperate formula
#for Accuracy and conditional, it give the user the ability to calculate their estimated probability from a test set.
def naiveBayes(chosen, table):
    return accuracy(conditional(chosen,table),table, chosen)


In [8]:
table = Table.read_table("bike_train.csv")
naiveBayes('count',table)

0.5806666666666667

In [9]:
table = Table.read_table("power_data.csv").drop(0,1)
naiveBayes('Power',table)

0.2791420666837797